In [1]:
pip install ipynb --upgrade

Requirement already up-to-date: ipynb in /Users/mattiaskallman1/anaconda3/lib/python3.7/site-packages (0.5.1)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pickle
import time
import os
import yaml
import sys
import pandas as pd

os.chdir("..")
import Models
os.chdir("MockPipeline")

os.chdir("../../pipeline")
import ipynb.fs.full.processing as processing
import ipynb.fs.full.features as features
import ipynb.fs.full.training as training
import ipynb.fs.full.analysis as analysis
import ipynb.fs.full.storage as storage
import ipynb.fs.full.visualize as visualize
os.chdir("../models-decision-system/MockPipeline")

In [3]:
os.getcwd()


'/Users/mattiaskallman1/Documents/Arcada/2020-2021/Decision_support_system/models-decision-system/MockPipeline'

In [4]:
def load_yaml(path):
    with open(path, mode='r') as file:
        return yaml.load(file, Loader=yaml.FullLoader)

In [5]:
settings = load_yaml('mock_settings.yaml')

In [6]:
settings['ensemble']['models'][0]

{'linreg': {'split': {'train': 0.8, 'test': 0.2}}}

In [7]:
dataframe = processing.create_dataframe(settings)

In [8]:
dataset = features.add(dataframe, settings['features'])

In [9]:
dataset

,Close,Momentum,SD,Label
Date_Timestamp,,,,
2010-02-01,1.39200,-0.04649,14.842308,1.39737
2010-02-02,1.39737,-0.03928,13.972612,1.38969
2010-02-03,1.38969,-0.05070,11.302307,1.37395
2010-02-04,1.37395,-0.05393,11.014429,1.36763
2010-02-05,1.36763,-0.04356,11.636518,1.36409
...,...,...,...,...
2019-12-25,1.10944,0.00301,54.215370,1.11012
2019-12-26,1.11012,0.00089,50.207241,1.11713
2019-12-27,1.11713,0.00388,49.232845,1.11813


In [10]:
data_linreg = features.split(dataset, 'linreg', settings['ensemble']['models'][0]['linreg'])

In [11]:
# WORKING
#linreg = Models.train_model(data_linreg, 'linreg', settings['ensemble']['models'][0])

In [12]:
data_lstm = features.split(dataset, 'lstm', settings['ensemble']['models'][1]['lstm'])

In [13]:
settings['ensemble']['models'][1]['lstm']['layers']

[{'lstm': {'value': 120, 'activation': 'relu'}},
 {'dropout': {'value': 0.15}},
 {'dense': {'value': 50, 'activation': 'relu'}},
 {'dense': {'value': 1}}]

In [14]:
# WORKING
lstm = Models.train_model(data_lstm, 'lstm', settings['ensemble']['models'][1]['lstm'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 1861 samples, validate on 620 samples
Epoch 1/7
1861/1861 [==============================] - 4s 2ms/sample - loss: 0.0282 - val_loss: 0.0155
Epoch 2/7
1861/1861 [==============================] - 2s 1ms/sample - loss: 0.0065 - val_loss: 0.0048
Epoch 3/7
1861/1861 [==============================] - 2s 1ms/sample - loss: 0.0051 - val_loss: 0.0025
Epoch 4/7
1861/1861 [==============================] - 2s 1ms/sample - loss: 0.0042 - val_loss: 0.0018
Epoch 5/7
1861/1861 [==============================] - 2s 1ms/sample - loss: 0.0031 - val_loss: 0.0014
Epoch 6/7
1861/1861 [==============================] - 2s 1ms/sample - loss: 0.0028 - val_loss: 0.0016
Epoch 7/7
1861/1861 [==============================] - 2s 1ms/sample - loss: 0.0025 - val_loss: 9.9606e-04


In [15]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.python.keras import Input, Model

from tcn import TCN, tcn_full_summary

batch_size, timesteps, input_dim = None, 20, 1


def get_x_y(size=1000):
    import numpy as np
    pos_indices = np.random.choice(size, size=int(size // 2), replace=False)
    x_train = np.zeros(shape=(size, timesteps, 1))
    y_train = np.zeros(shape=(size, 1))
    x_train[pos_indices, 0] = 1.0
    y_train[pos_indices, 0] = 1.0
    return x_train, y_train


i = Input(batch_shape=(batch_size, timesteps, input_dim))

o = TCN(return_sequences=False)(i)  # The TCN layers are here.
o = Dropout(0.15)(o)
o = Dense(50)(o)
o = Dense(1)(o)

m = Model(inputs=[i], outputs=[o])
m.compile(optimizer='adam', loss='mse')

tcn_full_summary(m, expand_residual_blocks=False)

x, y = get_x_y()
m.fit(x, y, epochs=10, validation_split=0.2)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20, 1)]           0         
_________________________________________________________________
residual_block_0 (ResidualBl [(None, 20, 64), (None, 2 8576      
_________________________________________________________________
residual_block_1 (ResidualBl [(None, 20, 64), (None, 2 16512     
_________________________________________________________________
residual_block_2 (ResidualBl [(None, 20, 64), (None, 2 16512     
_________________________________________________________________
residual_block_3 (ResidualBl [(None, 20, 64), (None, 2 16512     
_________________________________________________________________
residual_block_4 (ResidualBl [(None, 20, 64), (None, 2 16512     
_________________________________________________________________
residual_block_5 (ResidualBl [(None, 20, 64), (None, 2 16512 

In [16]:
#m.predict(x)

In [17]:
def add_base_layer_to_tcn(name, model_output, settings):
    available = {
        'dropout': Dropout,
        'dense': Dense
    }
    func = available[name]
    
    if name == 'dropout':
        return func(settings['value'])(model_output)
    else:
        return func(settings['value'])(model_output)
    
def add_tcn_layers(model_input, settings):
    model_output = TCN(return_sequences=False)(model_input)
    for index, layer in enumerate(settings['layers']):
        name = list(layer)[0]
        params = layer[name]
        if index == 0:
            continue
        else:
            model_output = add_base_layer_to_tcn(name, model_output, params)
    return model_output

In [18]:
test_input = Input(shape=(3, 1))
add_tcn_layers(test_input, settings['ensemble']['models'][3]['tcn'])

<tf.Tensor 'dense_5/BiasAdd:0' shape=(?, 1) dtype=float32>

In [19]:
def temporal_convolutional_network(data, settings):

    #  VARIABLES
    x_train = data['train']['features']
    y_train = data['train']['labels']
    x_validation = data['validation']['features']
    y_validation = data['validation']['labels']
    x_test = data['test']['features']
    scaler = data['scaler']
    timesteps = x_train.shape[0]
    input_dim = x_train.shape[1]
    batch_size = settings['batch']

    model_input = Input(shape=(input_dim, 1))

    model_output = add_tcn_layers(model_input, settings)
    
    model = Model(inputs=[model_input], outputs=[model_output])
    
    model.compile(optimizer='adam', loss='mse')

    tcn_full_summary(model, expand_residual_blocks=False)
    
    model.fit(x_train, y_train, epochs=settings['epochs'], validation_data=(x_validation, y_validation))
    #  https://keras.io/api/models/model_training_apis/#fit-method

    predictions = model.predict(x_test)

    return {
        'model': model,
        'predictions': predictions
    }

In [20]:
temporal_convolutional_network(data_lstm, settings['ensemble']['models'][3]['tcn'])

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 3, 1)]            0         
_________________________________________________________________
residual_block_0 (ResidualBl [(None, 3, 64), (None, 3, 8576      
_________________________________________________________________
residual_block_1 (ResidualBl [(None, 3, 64), (None, 3, 16512     
_________________________________________________________________
residual_block_2 (ResidualBl [(None, 3, 64), (None, 3, 16512     
_________________________________________________________________
residual_block_3 (ResidualBl [(None, 3, 64), (None, 3, 16512     
_________________________________________________________________
residual_block_4 (ResidualBl [(None, 3, 64), (None, 3, 16512     
_________________________________________________________________
residual_block_5 (ResidualBl [(None, 3, 64), (None, 3, 1651

{'model': <tensorflow.python.keras.engine.training.Model at 0x7fac39fe8210>,
 'predictions': array([[0.3171565 ],
        [0.29971454],
        [0.29224867],
        [0.30622032],
        [0.3401389 ],
        [0.3992496 ],
        [0.3927617 ],
        [0.41814846],
        [0.41278762],
        [0.38188505],
        [0.4028829 ],
        [0.40117282],
        [0.41461417],
        [0.41748056],
        [0.4340039 ],
        [0.45743445],
        [0.45505795],
        [0.4610857 ],
        [0.45513758],
        [0.42529637],
        [0.44164035],
        [0.4402213 ],
        [0.47789437],
        [0.46583438],
        [0.45619106],
        [0.43237436],
        [0.4403227 ],
        [0.41212842],
        [0.41462764],
        [0.40799224],
        [0.40057576],
        [0.41331112],
        [0.43897724],
        [0.476377  ],
        [0.481526  ],
        [0.44907263],
        [0.45260522],
        [0.45994905],
        [0.44349256],
        [0.43331242],
        [0.449226  ],
      

In [21]:
tcn = Models.train_model(data_lstm, 'tcn', settings['ensemble']['models'][3]['tcn'])

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 3, 1)]            0         
_________________________________________________________________
residual_block_0 (ResidualBl [(None, 3, 64), (None, 3, 8576      
_________________________________________________________________
residual_block_1 (ResidualBl [(None, 3, 64), (None, 3, 16512     
_________________________________________________________________
residual_block_2 (ResidualBl [(None, 3, 64), (None, 3, 16512     
_________________________________________________________________
residual_block_3 (ResidualBl [(None, 3, 64), (None, 3, 16512     
_________________________________________________________________
residual_block_4 (ResidualBl [(None, 3, 64), (None, 3, 16512     
_________________________________________________________________
residual_block_5 (ResidualBl [(None, 3, 64), (None, 3, 1651